In [1]:
import clean_database
import DataAugmentation
import Feature_Engineering as fe
import train_test_validation as ttv
import graficar
import linear
from linear import Linear_Model
import logistic
import mixture

#from mlp import MultilayerPerceptron
import numpy as np
import jax.numpy as jnp
import y_hot as hot
from mlp_mlflow import MultilayerPerceptron
import mlflow
import importlib

Limpieza de la base de datos
=====================================

In [13]:
datos_bajos,datos=clean_database.Clean_Data()

3364 7882


c:\Users\52333\Documents\doctorado\ml-md\proyecto\clean_database.py:375: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfd=d1.append(d2)


In [22]:
import pandas as pd
df=pd.read_csv('training_data_backwards_selection.csv')
X=df[df.columns[1:4]].to_numpy()
Y=df[df.columns[-1]].to_numpy()

Data augmentation para la clase de datos '1' (alumnos con bajo desempeño escolar)
=====================================================================================

In [14]:
datos_aumentados=DataAugmentation.Smote(datos_bajos)
print(datos_aumentados.shape)
datos=np.concatenate((datos,datos_aumentados),axis=0)
datos=datos[np.random.permutation(datos.shape[0]),:]
x,y=datos[:,1:],datos[:,0]

(3364, 96)


Feature Engineering (Backwards selection)
===============================================

In [15]:
x1,x2=fe.separate(x,y) #tenemos que separar los features en
print('estas son las muestras en casda clase',x1.shape[0],x2.shape[0])
size=x1.shape[1]
min_group=3 #son los features con los que nos queremos quedar
x1,x2,ganadores=fe.sequential_backwards(size,x1,x2,min_group)
x=x[:,ganadores]
x_train, x_test, x_v,y_train, y_test, y_v=ttv.split_data(x,y)

estas son las muestras en casda clase 7882 6728
los miembros del grupo ganador son (0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94)
los miembros del grupo ganador son (0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93)
los miembros del grupo ganador son (0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,

Multilayer Perceptron
======================================


In [1]:
x_trainp,y_trainp=jnp.transpose(x_train),jnp.transpose(y_train)
layers=[91,30,30,2]
lr=0.01
labels=jnp.array([0,1])
k_clases=2
samples=x.shape[0]
y_hotp=jnp.transpose(hot.one_hot(y_train,2))
stop=0.0001
max_steps=10000
x_valp,y_valp,y_hot_valp,samples_val=jnp.transpose(x_v),jnp.transpose(y_v),jnp.transpose(hot.one_hot(y_v,2)),x_v.shape[0]
run_name='mlp'
mlp_mlflow=MultilayerPerceptron(layers,lr,labels,k_clases,samples,x_trainp,y_trainp,y_hotp,stop,max_steps)
loss,recall_list,precision_list,loss_list,precision_list_val,recall_list_val,loss_list_val=mlp_mlflow.modelo(mlp_mlflow.weights,max_steps,x_trainp,y_trainp,y_hotp,lr,k_clases,samples,labels,stop,x_valp,y_valp,y_hot_valp,samples_val,run_name)


NameError: name 'jnp' is not defined

Linear Model
=================

In [ ]:
dim=int(x_train.shape[1])
y_trainl=y_train
y_vl=y_v

for i in range(9740):
    if y_train[i]==0:
        y_trainl[i]==-1
for i in range(1624):
    if y_v[i]==0:
        y_vl[i]=-1

x_trainl=jnp.array(x_train)
y_trainl=jnp.array(y_trainl)
x_vl=jnp.array(x_v)
y_vl=jnp.array(y_vl)
labels=jnp.array([0,1])
k_classes=2
n_steps=1000
lr=1e-6
samples=x_trainl.shape[0]
#linear model
#importlib.reload(linear)
modelo=Linear_Model(dim)
theta = modelo.generate_theta() #inicializamos theta con valores aleatorios
print(x_trainl.shape, y_trainl.shape)
y_trainl=jnp.reshape(y_trainl,(9740,1))
print(x_trainl.shape, y_trainl.shape,dim)
#theta = modelo.gradient_descent(theta, x_trainl, y_trainl, 10, lr = 0.01)
#y_hat=modelo.estimate_grsl(x_trainl,theta)





(9740, 90) (9740,)
(9740, 90) (9740, 1) 90


In [ ]:
run_name='lineal'
y_hatl,y_hat_vall=modelo.model(theta, x_trainl, y_trainl, lr,n_steps,x_vl,y_vl,run_name)

este es lr
este es el error en el paso i 0 43401.574
este es el error en el paso i 1 35594.105
este es el error en el paso i 2 32194.805
este es el error en el paso i 3 30336.92
este es el error en el paso i 4 29054.17
este es el error en el paso i 5 28017.623
este es el error en el paso i 6 27112.299
este es el error en el paso i 7 26295.375
este es el error en el paso i 8 25548.832
este es el error en el paso i 9 24862.883
este es el error en el paso i 10 24230.96
este es el error en el paso i 11 23648.262
este es el error en el paso i 12 23110.244
este es el error en el paso i 13 22613.117
este es el error en el paso i 14 22153.184
este es el error en el paso i 15 21727.514
este es el error en el paso i 16 21333.047
este es el error en el paso i 17 20967.111
este es el error en el paso i 18 20627.445
este es el error en el paso i 19 20311.713
este es el error en el paso i 20 20017.963
este es el error en el paso i 21 19744.246
este es el error en el paso i 22 19489.066
este es el er

KeyboardInterrupt: 

In [ ]:
#Ridge
l=0
importlib.reload(linear)

incremento=0.1
run_name='Ridge'
samples=500
x_train_r=x_train[:500]
y_train_r=y_train[:500]
x_vr=x_v[:500]
y_vr=y_v[:500]
y_hatR,y_hatR_v,precision_list=modelo.model_ridge(x_train_r,y_train_r,l,incremento,samples,k_classes,labels,x_vr,y_vr,run_name)



holaa


Logistic
================

In [62]:
importlib.reload(logistic)
from logistic import Logistic_Regression
x_train_log,y_train_log=jnp.transpose(x_train),jnp.transpose(y_train)
xlogv,ylogv=jnp.transpose(x_v),jnp.transpose(y_v)
model = Logistic_Regression()
alpha=1e-2
tol=1e-3

lbda=0.1
model.fit(x_train_log, y_train_log,alpha,tol,lbda)
Y_hat = model.estimate(x_train_log)
y_hat_v=model.estimate(xlogv)
run_name='logistic'
precision=model.precision(y_train_log, Y_hat,ylogv,y_hat_v,alpha,lbda,run_name)
print(precision)

0.1
lambda 0.1
-7713.833984375
-5637.595703125
-4684.83740234375
-4115.771484375
0.8593429327011108


Mixture of Gaussians
===========================

In [23]:
importlib.reload(mixture)
clases=int(2)


x11,x12=fe.separate(x_train,y_train)
print(x11.shape,x12.shape)
samples1=int(x11.shape[0])
samples2=int(x12.shape[0])
features=x12.shape[1]
print(features)

#valores para inicializar el paso EM
p=np.array([0.3,0.7])
mu1=np.array([0.9,0.4,0.4])
mu2=np.array([0.5,0.8,0.4])
sigma1=np.identity(features)
sigma2=np.identity(features)
#sigma1=([0.1,0.2],[0.3,0.4])
#sigma2=([0.4,0.5],[0.6,0.8])
mu1,sigma1,mu2,sigma2,p,Nk1,Nk2=mixture.solve(clases,features,samples1,samples2,p,mu1,mu2,sigma1,sigma2,x11,x12)
samples=x_v.shape[0]
y_hat,precision=mixture.get_prediction(x_v,mu1,sigma1,mu2,sigma2,features,samples,y_v)
precision

NameError: name 'x_train' is not defined

In [26]:
clases=int(2)

importlib.reload(mixture)
x11=X[:27940]
x12=X[27940:]
print(x11.shape,x12.shape)
samples1=int(x11.shape[0])
samples2=int(x12.shape[0])
features=x12.shape[1]
print(x11)
print('este es samples1',samples1)

#valores para inicializar el paso EM
p=np.array([0.3,0.7])
mu1=np.array([0.9,0.4,0.4])
mu2=np.array([0.5,0.8,0.4])
sigma1=np.identity(features)
sigma2=np.identity(features)
#sigma1=([0.1,0.2],[0.3,0.4])
#sigma2=([0.4,0.5],[0.6,0.8])
mu1,sigma1,mu2,sigma2,p,Nk1,Nk2=mixture.solve(clases,features,samples1,samples2,p,mu1,mu2,sigma1,sigma2,x11,x12)
samples=x_v.shape[0]
y_hat,precision=mixture.get_prediction(x_v,mu1,sigma1,mu2,sigma2,features,samples,y_v)
precision

(27940, 3) (27940, 3)
[[-1.06055587 -1.16189943 -0.06724547]
 [-0.83942897 -1.03339214 -0.06724547]
 [-1.11583759 -1.12201786 -0.1083073 ]
 ...
 [-0.33660888 -0.92442604  0.7013484 ]
 [-1.38785448 -1.58951624 -0.28613056]
 [-1.90632798 -1.54012099 -0.72212716]]
este es samples1 27940
1
estos son los nk 8209.818536853043 19786.66594807832
[-0.62525737 -0.68021634  0.47797682] [[0.19274048 0.1255379  0.01479961]
 [0.1255379  0.20406945 0.01748083]
 [0.01479961 0.01748083 0.74813048]] [ 0.63080049  0.66805972 -0.47991594] [[ 0.97364537  0.8520329  -0.47469061]
 [ 0.8520329   0.92897417 -0.50652782]
 [-0.47469061 -0.50652782  0.79786355]] [0.14691873 0.35409209]
2
estos son los nk 18203.97366130002 23310.378350906936
[-0.6857685  -0.73164413  0.5713924 ] [[0.14706074 0.09376704 0.04812734]
 [0.09376704 0.16522518 0.06498905]
 [0.04812734 0.06498905 0.83758839]] [ 0.83772922  0.86078527 -0.62299717] [[ 0.9113111   0.77206993 -0.41216269]
 [ 0.77206993  0.83968574 -0.44169303]
 [-0.41216269 

NameError: name 'x_v' is not defined